In [ ]:
!git -C ColBERT/ pull || git clone https://github.com/stanford-futuredata/ColBERT.git
import sys; sys.path.insert(0, 'ColBERT/')


fatal: cannot change to 'ColBERT/': No such file or directory
Cloning into 'ColBERT'...
remote: Enumerating objects: 2576, done.
remote: Counting objects: 100% (1079/1079), done.
remote: Compressing objects: 100% (337/337), done.
remote: Total 2576 (delta 843), reused 786 (delta 742), pack-reused 1497
Receiving objects: 100% (2576/2576), 2.02 MiB | 3.84 MiB/s, done.
Resolving deltas: 100% (1602/1602), done.


In [ ]:
try: # When on google Colab, let's install all dependencies with pip.
    import google.colab
    !pip install -U pip
    !pip install -e ColBERT/['faiss-gpu','torch']
except Exception:
  import sys; sys.path.insert(0, 'ColBERT/')
  try:
    from colbert import Indexer, Searcher
  except Exception:
    print("If you're running outside Colab, please make sure you install ColBERT in conda following the instructions in our README. You can also install (as above) with pip but it may install slower or less stable faiss or torch dependencies. Conda is recommended.")
    assert False

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Obtaining file:///content/ColBERT
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/3

In [ ]:
import colbert
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection

In [ ]:
#Get queries and collection

!wget "https://drive.google.com/uc?export=download&id=1IQixcaiOm8CayM5Lt9EkidKS4a608DFo" -O "colBERT_queries"
!wget "https://drive.google.com/uc?export=download&id=1BJCpMx9yk41wsrFMB0AW4albLDZd5z06" -O "colBERT_docs"


--2024-01-24 17:58:37--  https://drive.google.com/uc?export=download&id=1IQixcaiOm8CayM5Lt9EkidKS4a608DFo
Resolving drive.google.com (drive.google.com)... 74.125.24.102, 74.125.24.113, 74.125.24.138, ...
Connecting to drive.google.com (drive.google.com)|74.125.24.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1IQixcaiOm8CayM5Lt9EkidKS4a608DFo&export=download [following]
--2024-01-24 17:58:38--  https://drive.usercontent.google.com/download?id=1IQixcaiOm8CayM5Lt9EkidKS4a608DFo&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 172.253.118.132, 2404:6800:4003:c05::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|172.253.118.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1862 (1.8K) [application/octet-stream]
Saving to: ‘colBERT_queries’

colBERT_queries     100%[===================>]   1.82K  --.-KB/s    

In [ ]:
import json

with open('colBERT_docs', "r") as file:
  docs = json.load(file)

with open('colBERT_queries', "r") as file:
  queries = json.load(file)


In [ ]:
doc_ids = list(docs.keys())
docs = list(docs.values())

query_ids = list(queries.keys())
queries = list(queries.values())

In [ ]:
nbits = 2   # encode each dimension with 2 bits

doc_maxlen = 400 # truncate passages at 400 tokens

In [ ]:
#Create indexer

checkpoint = 'colbert-ir/colbertv2.0'

with Run().context(RunConfig(nranks=1, experiment='notebook')):  # nranks specifies the number of GPUs to use
    config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits, kmeans_niters=30) # kmeans_niters specifies the number of iterations of k-means clustering; 4 is a good and fast default.
                                                                                # Consider larger numbers for small datasets.

    indexer = Indexer(checkpoint=checkpoint, config=config)
    indexer.index(name='index', collection=docs, overwrite=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]



[Jan 24, 17:58:43] #> Creating directory /content/experiments/notebook/indexes/index 


#> Starting...
#> Joined...


In [ ]:
#Create searcher

with Run().context(RunConfig(experiment='notebook')):
    searcher = Searcher(index='index', collection=docs)

[Jan 24, 18:02:29] #> Loading codec...
[Jan 24, 18:02:29] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Jan 24, 18:02:29] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Jan 24, 18:02:30] #> Loading IVF...
[Jan 24, 18:02:30] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 3363.52it/s]

[Jan 24, 18:02:30] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 109.01it/s]


In [ ]:

#Get results for all queries
results=[]

for query in queries:

  result = searcher.search(query, k=500)


  answer=[]
  for passage_id, passage_rank, passage_score in zip(*result):
      answer.append(int(doc_ids[passage_id]))

  results.append(answer)


#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . WHAT ARE THE EFFECTS OF CALCIUM ON THE PHYSICAL PROPERTIES OF MUCUS FROM CF PATIENTS, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2054,  2024,  1996,  3896,  1997, 13853,  2006,  1996,
         3558,  5144,  1997, 14163,  7874,  2013, 12935,  5022,   102,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')



In [ ]:
from google.colab import files
import pickle

with open("colbert_results", "wb") as file:
  pickle.dump(results, file)

files.download("colbert_results")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>